In [45]:
from sknn.mlp import Classifier, Convolution, FastVectorSpace, Layer, MultiLayerPerceptron
from sknn.mlp import NeuralNetwork, Regressor, SparseDesignMatrix
from time import time
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.grid_search import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.externals import joblib
from sklearn import datasets
from random import randint

In [2]:
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

In [3]:
def XOR(n):
    # 1000개의 데이터를 생성
    X = [[randint(0, 1), randint(0, 1)] for _ in range(n)]
    y = []
    for a, b in X:
        if a == b:
            y.append([0])
        else:
            y.append([1])
    X = np.array(X)
    Y = np.array(y)
    return X, Y

In [45]:
features, labels = XOR(n=1000)
(trainX, testX, trainY, testY) = train_test_split(features, labels, test_size = 0.3)

In [46]:
print "the shape of training set %s rows, %s columns" %(trainX.shape[0], trainX.shape[1])
print "the shape of test set %s rows, %s columns" %(testX.shape[0], testX.shape[1])
print "the range of training set : %s ~ %s" %(trainX.min(),trainX.max())
print "the range of test set : %s ~ %s" %(testX.min(),testX.max())

the shape of training set 700 rows, 2 columns
the shape of test set 300 rows, 2 columns
the range of training set : 0 ~ 1
the range of test set : 0 ~ 1


In [92]:
# Regression Example
nn = Regressor(
    layers=[
        Layer("Rectifier", units=100),
        Layer("Linear")],
    learning_rate=0.2,
    n_iter=100)
nn.fit(trainX, trainY)

# compute the predictions for the test data and show a classification report
preds = nn.predict(testX)
preds = preds.astype('int64')

print "accuracy score : %s" %(accuracy_score(testY, preds))
print "classification report : "
print classification_report(testY, preds)

accuracy score : 1.0
classification report : 
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       148
          1       1.00      1.00      1.00       152

avg / total       1.00      1.00      1.00       300



In [93]:
# Classification 
nn = Classifier(
    layers=[
        Layer("Maxout", units=100, pieces=2),
        Layer("Softmax")],
    learning_rate=0.001,
    n_iter=25)
nn.fit(trainX, trainY)

# compute the predictions for the test data and show a classification report
preds = nn.predict(testX)
preds = preds.astype('int64')

print "accuracy score : %s" %(accuracy_score(testY, preds))
print "classification report : "
print classification_report(testY, preds)

accuracy score : 1.0
classification report : 
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       148
          1       1.00      1.00      1.00       152

avg / total       1.00      1.00      1.00       300



In [139]:
dataset = datasets.fetch_mldata("MNIST Original", data_home=".")

features = np.array(dataset.data, 'float32') / 255
labels = np.array(dataset.target, 'int')

(trainX, testX, trainY, testY) = train_test_split(features, labels, test_size = 0.3)

# reshape for convolutions
trainX = trainX.reshape((trainX.shape[0], 1, 28, 28))
testX = testX.reshape((testX.shape[0], 1, 28, 28))

In [142]:
print "the shape of training set %s rows, %s columns" %(trainX.shape[0], trainX.shape[1])
print "the shape of test set %s rows, %s columns" %(testX.shape[0], testX.shape[1])
print "the range of training set : %s ~ %s" %(trainX.min(),trainX.max())
print "the range of test set : %s ~ %s" %(testX.min(),testX.max())

the shape of training set 49000 rows, 1 columns
the shape of test set 21000 rows, 1 columns
the range of training set : 0.0 ~ 1.0
the range of test set : 0.0 ~ 1.0


In [149]:
# Convolution 
nn = Classifier(
    layers=[
        Convolution("Rectifier", channels=9, kernel_shape=(3,3), border_mode='full'),
        Layer("Softmax")],
    learning_rate=0.02,
    n_iter=10)
nn.fit(trainX, trainY)

# compute the predictions for the test data and show a classification report
preds = nn.predict(testX)

print "accuracy score : %s" %(accuracy_score(testY, preds))
print "classification report : "
print classification_report(testY, preds)

accuracy score : 0.97119047619
classification report : 
             precision    recall  f1-score   support

          0       0.97      0.99      0.98      2026
          1       0.98      0.99      0.98      2331
          2       0.97      0.97      0.97      2128
          3       0.97      0.97      0.97      2183
          4       0.94      0.98      0.96      1976
          5       0.98      0.97      0.97      1905
          6       0.98      0.98      0.98      2085
          7       0.96      0.98      0.97      2181
          8       0.98      0.95      0.97      2031
          9       0.98      0.92      0.95      2154

avg / total       0.97      0.97      0.97     21000



In [24]:
# Grid Search
# "Rectifier", "Sigmoid", "Tanh", "Maxout", "Linear", "Softmax", "Gaussian"
features, labels = XOR(n=100)
(trainX, testX, trainY, testY) = train_test_split(features, labels, test_size = 0.3)

nn = Regressor(
    layers=[
        Layer("Sigmoid", units=10),# 첫번째 히든레이어
        Layer("Sigmoid", units=10),# 두번째 히든레이어
        Layer("Linear")], # 아웃풋 레이어
    verbose=1)

gs = GridSearchCV(nn, param_grid={
    'learning_rate': [0.05, 0.01],
    'n_iter' : [5],
    'hidden0__units': [20, 30],
    'hidden0__type': ["Sigmoid", "Tanh"], # 첫번째 히든레이어
    'hidden1__units': [20, 30],
    'hidden1__type': ["Sigmoid", "Tanh"]}) # 두번째 히든레이어
gs.fit(trainX, trainY)

GridSearchCV(cv=None,
       estimator=Regressor(batch_size=1, debug=False, dropout_rate=None, f_stable=0.001,
     hidden0=<sknn.nn.Layer `Sigmoid`: name=u'hidden0', units=10>,
     hidden1=<sknn.nn.Layer `Sigmoid`: name=u'hidden1', units=10>,
     layers=[<sknn.nn.Layer `Sigmoid`: name=u'hidden0', units=10>, <sknn.nn.Layer `S...state=None,
     regularize=None, valid_set=None, valid_size=0.0, verbose=1,
     weight_decay=None),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'n_iter': [5], 'hidden0__units': [20, 30], 'learning_rate': [0.05, 0.01], 'hidden1__units': [20, 30], 'hidden1__type': ['Sigmoid', 'Tanh'], 'hidden0__type': ['Sigmoid', 'Tanh']},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)

In [25]:
print gs.best_params_
print 
print gs.best_estimator_

{'n_iter': 5, 'hidden0__units': 30, 'learning_rate': 0.05, 'hidden1__units': 30, 'hidden1__type': 'Sigmoid', 'hidden0__type': 'Tanh'}

Regressor(batch_size=1, debug=False, dropout_rate=None, f_stable=0.001,
     hidden0=<sknn.nn.Layer `Tanh`: name=u'hidden0', units=30>,
     hidden1=<sknn.nn.Layer `Sigmoid`: name=u'hidden1', units=30>,
     layers=[<sknn.nn.Layer `Tanh`: name=u'hidden0', units=30>, <sknn.nn.Layer `Sigmoid`: name=u'hidden1', units=30>, <sknn.nn.Layer `Linear`: name=u'output', units=1>],
     learning_momentum=0.9, learning_rate=0.05, learning_rule=u'sgd',
     loss_type=u'mse', n_iter=5, n_stable=50,
     output=<sknn.nn.Layer `Linear`: name=u'output', units=1>,
     random_state=None, regularize=None, valid_set=None, valid_size=0.0,
     verbose=1, weight_decay=None)


In [26]:
# compute the predictions for the test data and show a classification report
preds = gs.predict(testX)
preds = preds.astype('int64')


print "accuracy score : %s" %(accuracy_score(testY, preds))
print "classification report : "
print classification_report(testY, preds)

accuracy score : 0.566666666667
classification report : 
             precision    recall  f1-score   support

          0       0.57      1.00      0.72        17
          1       0.00      0.00      0.00        13

avg / total       0.32      0.57      0.41        30



In [42]:
dataset = datasets.fetch_mldata("MNIST Original", data_home=".")

features = np.array(dataset.data, 'float32') / 255
labels = np.array(dataset.target, 'int')

(trainX, testX, trainY, testY) = train_test_split(features, labels, test_size = 0.3)

# reshape for convolutions
trainX = trainX.reshape((trainX.shape[0], 1, 28, 28))
testX = testX.reshape((testX.shape[0], 1, 28, 28))

# Convolution 
nn = Classifier(
    layers=[
        Convolution("Rectifier", channels=9, kernel_shape=(3,3), border_mode='full'),
        Convolution("Rectifier", channels=9, kernel_shape=(2,2), border_mode='full'),
        Layer("Softmax")],
    learning_rate=0.02,
    n_iter=10)
nn.fit(trainX, trainY)

# compute the predictions for the test data and show a classification report
preds = nn.predict(testX)

print "accuracy score : %s" %(accuracy_score(testY, preds))
print "classification report : "
print classification_report(testY, preds)

accuracy score : 0.966476190476
classification report : 
             precision    recall  f1-score   support

          0       0.97      0.99      0.98      2119
          1       0.98      0.99      0.99      2382
          2       0.97      0.93      0.95      2068
          3       0.94      0.97      0.95      2091
          4       0.97      0.98      0.97      2023
          5       0.96      0.96      0.96      1909
          6       0.98      0.98      0.98      2052
          7       0.96      0.97      0.97      2194
          8       0.97      0.95      0.96      2024
          9       0.96      0.95      0.95      2138

avg / total       0.97      0.97      0.97     21000



In [43]:
nn

Classifier(batch_size=1, debug=False, dropout_rate=None, f_stable=0.001,
      hidden0=<sknn.nn.Convolution `Rectifier`: channels=9, name=u'hidden0', kernel_shape=(3, 3), kernel_stride=(1, 1), pool_shape=(1, 1), border_mode='full'>,
      hidden1=<sknn.nn.Convolution `Rectifier`: channels=9, name=u'hidden1', kernel_shape=(2, 2), kernel_stride=(1, 1), pool_shape=(1, 1), border_mode='full'>,
      layers=[<sknn.nn.Convolution `Rectifier`: channels=9, name=u'hidden0', kernel_shape=(3, 3), kernel_stride=(1, 1), pool_shape=(1, 1), border_mode='full'>, <sknn.nn.Convolution `Rectifier`: channels=9, name=u'hidden1', kernel_shape=(2, 2), kernel_stride=(1, 1), pool_shape=(1, 1), border_mode='full'>, <sknn.nn.Layer `Softmax`: name=u'output', units=10>],
      learning_momentum=0.9, learning_rate=0.02, learning_rule=u'sgd',
      loss_type=u'mse', n_iter=10, n_stable=50,
      output=<sknn.nn.Layer `Softmax`: name=u'output', units=10>,
      random_state=None, regularize=None, valid_set=None, vali

In [44]:
testY

array([0, 1, 3, ..., 8, 9, 4])

In [54]:
from sknn.ae import AutoEncoder, Layer
# 위에서와 Layer가 이름이 겹치므로 주의

dataset = datasets.fetch_mldata("MNIST Original", data_home=".")

features = np.array(dataset.data, 'float32') / 255
labels = np.array(dataset.target, 'int')

(trainX, testX, trainY, testY) = train_test_split(features, labels, test_size = 0.3)

# Regression Example
nn = AutoEncoder(
    layers=[
        Layer("Sigmoid", units=100),
        Layer("Tanh", units=10)],
    learning_rate=0.2,
    n_iter=10)
nn.fit(trainX)

In [65]:
print nn.layers
print nn.f_stable

[<sknn.nn.Layer `autoencoder`: tied_weights=True, cost=u'msre', corruption_level=0.5, name=u'hidden0', units=100, activation='sigmoid'>, <sknn.nn.Layer `autoencoder`: tied_weights=True, cost=u'msre', corruption_level=0.5, name=u'output', units=10, activation='tanh'>]
0.001


In [82]:
print testX.shape
print nn.transform(testX).shape # "Tanh", units=10 이라서 아웃풋이 10으로 나온거임

(21000, 784)
(21000, 10)


In [84]:
print np.unique(nn.transform(testX))
print len(np.unique(nn.transform(testX)))

[-1. -1. -1. ...,  1.  1.  1.]
95101


In [81]:
print nn.is_convolution
print nn.learning_momentum
print nn.learning_rate
print nn.learning_rule
print nn.loss_type
print nn.n_iter
print nn.n_stable
print nn.random_state
print nn.regularize
print nn.valid_set
print nn.weight_decay

False
0.9
0.2
sgd
mse
10
50
None
None
None
None


In [37]:
# 왜 안될까?
# Grid Search
# "Rectifier", "Sigmoid", "Tanh", "Maxout", "Linear", "Softmax", "Gaussian"
features, labels = XOR(n=100)
(trainX, testX, trainY, testY) = train_test_split(features, labels, test_size = 0.3)

nn = Classifier(
    layers=[
        Layer("Maxout", units=100, pieces=2),
        Layer("Softmax")],
    learning_rate=0.001,
    n_iter=5)

# nn.fit(trainX, trainY)

gs = GridSearchCV(nn, param_grid={'learning_rate': [0.05, 0.01]})
gs.fit(trainX, trainY)

# gs = GridSearchCV(nn, param_grid={
#     'learning_rate': [0.05, 0.01],
#     'n_iter' : [5],
#     'pieces' : [2]}) # 두번째 히든레이어
# gs.fit(trainX, trainY)

# # compute the predictions for the test data and show a classification report
# preds = gs.predict(testX)
# preds = preds.astype('int64')


# print "accuracy score : %s" %(accuracy_score(testY, preds))
# print "classification report : "
# print classification_report(testY, preds)

IndexError: too many indices for array

In [39]:
print trainY.dtype
print testY.dtype

int64
int64
